# 데이터 불러오기

In [4]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv("recipe3.csv")
df["x"] = df["x"].apply(literal_eval)
df["x"] = df["x"].apply(lambda x: ', '.join(x))
df["recipeIngredient"] = df["recipeIngredient"].apply(literal_eval)
df["recipeInstructions"] = df["recipeInstructions"].apply(literal_eval)

## 라벨 생성 (레시피명 + 조리과정)

In [5]:
df["y"] = df.apply(lambda row: row["name"]+"\n\n"+'\n'.join(row["recipeInstructions"]), axis=1)
df = df[["x","y"]]
df.head()

,x,y
0,"통 삼겹, 청경채, 대파, 양파, 통마늘, 생강, 통후추, 월계수잎, 진간장, 굴 ...",(집에서 즐기는 술안주) 동파육 - 통삼겹 요리 / 홈파티 요리\n\n홈 파티 요리...
1,"당근, 양파, 표고버섯, 목이버섯, 어묵, 호박고지, 돼지고기 잡채용, 시금치나물,...",#잡채만들기 #간단하게 만드는 잡채 #한꺼번에 볶아내면 금방 뚝딱 만드는 잡채!!!...
2,"닭볶음용 닭, 양파, 당근, 감자, 대파, 고추장, 다진 마늘, 맛술, 고춧가루, ...",푹 익힌 닭에 깊은 양념맛이 좋은 닭볶음탕\n\n요즘 마트에가면 닭볶음용 온가족세트...
3,"배추, 양파, 사과, 배, 당근, 무, 쪽파, 홍고추, 골드 키위, 소금, 새우젓,...",맵지않아 더욱더 맛나다는 백김치의 비법은?\n\n배추 2포기를 4등분으로 잘라주신후...
4,"메밀가루, 계란, 우유, 3층 치즈 2장, 아몬드 슬라이스, 건 크랜베리, 강황가루...",메밀부꾸미\n\n껍질벗긴 메밀3T를 분쇄기로 갈아요\n (시판 메밀가루 3T 쓰셔도...


# 모델 학습

In [ ]:
pip install torch

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import torch

# 학습용 입력, 출력 데이터 생성
inputs = df["x"].values
outputs = df["y"].values

# T5 모델과 토크나이저 로딩
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 인코딩 및 패딩
input_ids = tokenizer.batch_encode_plus(inputs, padding=True, max_length=128, return_tensors="pt")["input_ids"]
output_ids = tokenizer.batch_encode_plus(outputs, padding=True, max_length=128, return_tensors="pt")["input_ids"]

# 학습 인스턴스 생성
train_dataset = torch.utils.data.TensorDataset(input_ids, output_ids)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 모델 체크포인트와 예측 결과를 저장할 경로
    num_train_epochs=3,              # 전체 학습 에폭 수
    per_device_train_batch_size=8,   # 한번의 학습에 사용할 배치 크기
    per_device_eval_batch_size=8,    # 한번의 평가에 사용할 배치 크기
    warmup_steps=500,                # learning rate warmup step 수
    weight_decay=0.01,               # 가중치 감소 비율
    logging_dir='./logs',            # 학습 로그 저장 경로
    logging_steps=500,               # 학습 로그 저장 간격
    eval_steps=500,                  # 평가 스텝 간격
    save_total_limit=5               # 저장할 체크포인트 파일 수
)

# 훈련 인스턴스 생성 및 학습 시작
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
trainer.train()

---

# 사전학습 모델 테스트

In [45]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# load the pre-trained KoGPT tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2')

# load the pre-trained KoGPT model
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# generate text using the model
input_text = """
'코코넛설탕', '토막난 닭', '말린 가지', '멸치다시마육수'
위 재료를 사용하는 요리명과 레시피는 다음과 같습니다.

"""
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids, max_length=150, do_sample=True, temperature=0.3)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.



'코코넛설탕', '토막난 닭', '말린 가지', '멸치다시마육수'
위 재료를 사용하는 요리명과 레시피는 다음과 같습니다.

1. 닭가슴살
2. 닭가슴살
3. 닭가슴살
4. 닭가슴살
5. 닭가슴살
6. 닭가슴살
7. 닭가슴살
8. 닭가슴살
9. 닭가슴살
10. 닭가슴살
11. 닭가슴살
12. 닭가슴살
13. 닭가슴살
14. 닭가슴살
15. 닭가슴살
16. 닭가슴살
17. 닭가슴살
19. 닭가슴살
20. 닭가슴살
20. 닭가슴
